In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
    output_df = input_df.withColumn("ingestion_date", current_timestamp())
    return output_df

In [0]:
#En caso de que se ejecute el mismo Notebook, con las misma fecha del file_date, hay que evitar que se dupliquen los datos,
# Para eso podemos eliminar la particion.
#Por ejemplo si ejecutamos la particion 2024-12-23, dos veces, cuando se ejecuta por segunda vez, elimnamos la particion existente. Esto evita duplicados.

#collect, se utiliza para cargar los datos en memoria
def overwrite_partition(input_df,db_name,table_name,column_partition):
    for item_list in input_df.select(f"{column_partition}").distinct().collect():
        if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")): #Si la tabla movies existe,ejecuta la siguiente instruccion
            spark.sql(f"ALTER TABLE {db_name}.{table_name} DROP IF EXISTS PARTITION ({column_partition} = '{item_list[column_partition]}')")

In [0]:

# Cuando se tengan coincidencias en "tgt.movie_id = src.movie_id"" (whenMatchedUpdateAll()). Pues actuliza todos los valores de las columnas. Es decir en la tabla tgt se va a proceder a actualizar todos los registros en base a la tabla de origen (movies_final_df).

# Si no hay concidencias (whenNotMatchedInsertAll()), pues se insertan todos los registros en la tabla tgt.
# Es decir, se ingresar los registros de movies_final_df a la tabla tgt.
from delta.tables import DeltaTable

def merge_delta_lake(input_df,db_name,table_name,folder_path, merge_condition, partition_column):

    if(spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):

        deltaTable = DeltaTable.forPath(spark,f"{folder_path}/{table_name}")

        deltaTable.alias("tgt") \
            .merge(
                input_df.alias("src"),
                merge_condition # Buscar por particion
            ) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")